In [7]:
import pandas as pd
import numpy as np
import sys
import os

sys.path.append(os.path.abspath('..'))
from processamento import df

In [13]:
# Definir as doenças respiratórias
respiratory_conditions = ['Asthma', 'Chronic obstructive pulmonary disease']

# Criar o novo atributo RespDisease: 1 se for doença respiratória, 0 caso contrário
df['RespDisease'] = df['Outcome'].apply(lambda x: 1 if x in respiratory_conditions else 0)

# Contagem específica
asthma_count = df['Outcome'].value_counts().get('Asthma', 0)
copd_count = df['Outcome'].value_counts().get('Chronic obstructive pulmonary disease', 0)

print(f"Asthma: {asthma_count}")
print(f"Chronic obstructive pulmonary disease: {copd_count}")

# Confirmar com RespDisease
print(f"Doenças respiratórias: {(df['RespDisease'] == 1).sum()}")


Asthma: 10254
Chronic obstructive pulmonary disease: 8485
Doenças respiratórias: 18739
